In [1]:
!pip install rdp
!pip install fastdtw
!pip install pactus

  Preparing metadata (setup.py) ... done
  Created wheel for rdp: filename=rdp-0.8-py3-none-any.whl size=4585 sha256=c9e5d280e071633701ea469e03d11f876da9c306a13c8b029592f2af891b481a
  Stored in directory: /root/.cache/pip/wheels/5d/12/ec/0fc50553af000b9c3d2c74b9f77a01ae4bfe856e9917ac239c
Successfully built rdp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp310-cp310-linux_x86_64.whl size=512549 sha256=d264a1dd441a1afc8fa90d27fd6a90add804e6be4bb9c502aca7089e6a6d198d
  Stored in directory: /root/.cache/pip/wheels/73/c8/f7/c25448dab74c3acf4848bc25d513c736bb93910277e1528ef4
Successfully built fastdtw
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 1.3 MB/s eta 0:00:00


In [2]:
import keras

In [3]:
import os
import csv
import hashlib
import random
import itertools
import numpy as np
import psutil
from datetime import datetime
from itertools import chain
from typing import List, Tuple, Any

import geopandas as gpd
from shapely.geometry import LineString, Point
from fastdtw import fastdtw
from rdp import rdp
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Ridge, LogisticRegression, LinearRegression

from pactus import Dataset, featurizers
from yupi import Trajectory
from pactus.models import (
    DecisionTreeModel,
    KNeighborsModel,
    LSTMModel,
    RandomForestModel,
    SVMModel,
    TransformerModel,
    XGBoostModel,
)

# Classification Model

## Segmentation

In [4]:
def rdp_segmentation(traj: list, epsilon=0.005):
    """Segment trajectory using the RDP algorithm while ensuring reduced points are used only once and no overlap occurs."""
    try:
        # Apply the RDP algorithm to get reduced points
        reduced_points = rdp(traj, epsilon=epsilon)
        
        # Convert reduced points to a set for easy lookup
        reduced_points_set = set(map(tuple, reduced_points))
        
        # Create segments based on reduced points
        segments = []
        current_segment = []
        
        # Track the index of the last used reduced point
        last_used_index = -1
        
        for i, point in enumerate(traj):
            current_segment.append(point)
            
            # If the point is a reduced point and it hasn't been used yet, create a segment
            if tuple(point) in reduced_points_set and i > last_used_index:
                segments.append(current_segment)
                current_segment = []
                last_used_index = i  # Update the index of the last used reduced point
        
        # Append the last segment if it exists
        if current_segment:
            segments.append(current_segment)
        
        return segments
    except Exception as e:
        print(f"Error: {e}")
        return None


def random_segmentation(traj, num_segments=4):
    """Segment trajectory into a random number of segments."""
    indices = sorted(random.sample(range(1, len(traj)), num_segments - 1))
    segments = [traj[i:j] for i, j in zip([0] + indices, indices + [len(traj)])]
    return segments


def sliding_window_segmentation(traj, step=5, percentage=5):
    """
    !!! Step===percentage
    """
    if not 0 < percentage <= 100:
        raise ValueError("Percentage must be between 0 and 100.")

    window_size = max(1, int((percentage / 100) * len(traj)))
    segments = [traj[i:i + window_size] for i in range(0, len(traj) - window_size + 1, step)]
    return segments

def mdl_cost(segment):
    """Tính chi phí MDL cho một segment."""
    if len(segment) < 2:
        return float('inf')  # Không thể mô tả với ít hơn 2 điểm

    start, end = segment[0], segment[-1]
    segment_array = np.array(segment)
    line = np.array(end) - np.array(start)

    if np.linalg.norm(line) == 0:
        return float('inf')  # Tránh chia cho 0 nếu start == end

    normalized_line = line / np.linalg.norm(line)
    projections = np.dot(segment_array - start, normalized_line)
    reconstruction = start + np.outer(projections, normalized_line)
    error = np.linalg.norm(segment_array - reconstruction, axis=1).sum()

    return error

def mdl_segmentation(traj, epsilon=0.8):
    """Phân đoạn trajectory sử dụng thuật toán MDL."""
    segments = []
    current_segment = [traj[0]]

    for i in range(1, len(traj)):
        current_segment.append(traj[i])
        if mdl_cost(current_segment) > epsilon:
            # Chốt đoạn trước
            segments.append(current_segment[:-1])
            current_segment = [traj[i]]

    # Thêm đoạn cuối
    if current_segment:
        segments.append(current_segment)

    return segments

## Pertubation

In [5]:
def gaussian_perturbation(segment, mean=0, std=3, scale=1.5):
    new_segment = []
    for point in segment:
        x, y = point
        new_x = x + np.random.normal(mean, std) * scale
        new_y = y + np.random.normal(mean, std) * scale
        new_segment.append((new_x, new_y))
    return new_segment

def scaling_perturbation(segment, scale_factor=1.2):
    new_segment = []
    for point in segment:
        x, y = point
        new_x = x * scale_factor
        new_y = y * scale_factor
        new_segment.append((new_x, new_y))
    return new_segment

def rotation_perturbation(segment, angle=np.pi/18):
    new_segment = []
    cos_angle = np.cos(angle)
    sin_angle = np.sin(angle)
    for point in segment:
        x, y = point
        new_x = x * cos_angle - y * sin_angle
        new_y = x * sin_angle + y * cos_angle
        new_segment.append((new_x, new_y))
    return new_segment

## XAI to get explain

In [6]:
from fastdtw import fastdtw
from sklearn.linear_model import Ridge
from scipy.spatial.distance import euclidean
import random

class TrajectoryManipulator:
    def __init__(self, X, segmentation_model, perturbation_model, model, deep_learning_model=False):
        try:
            self.X = list(X)  # Chuyển về list thay vì numpy array
            
            self.segmentation_model = segmentation_model
            self.perturbation_model = perturbation_model
            self.model = model
            self.deep_learning_model = deep_learning_model
            
            try:
                self.segments = self._segmentation(self.X)
                print(f"Segments shape: {len(self.segments)}, Segments data: {self.segments}")
            except Exception as e:
                print(f"Error in _segmentation: {e}")
            
            self.x_len = len(self.segments)
            self.number_of_permutations = min(2 ** 10, 2 ** self.x_len)
        
            self.perturb_vectors = self.create_perturbation_points_by_shuffle(self.x_len, self.number_of_permutations)
            
            try:
                self.clean_segments = self.segments
                self.noisy_segments = [self._perturbation(seg) for seg in self.segments]
                print(f"noise: ")
            except Exception as e:
                print(f"Error in creating noisy segments: {e}")
            
            try:
                self.Z_eval = self._createZForEval()
            except Exception as e:
                print(f"Error in _createZForEval: {e}")
        
        except Exception as e:
            print(f"Error in __init__: {e}")

    def _segmentation(self, points_list):
        try:
            return self.segmentation_model(points_list)
        except Exception as e:
            print(f"Error in _segmentation: {e}")
            return []

    def _perturbation(self, segment):
        try:
            perturbed_segment = self.perturbation_model(segment)
            return perturbed_segment
        except Exception as e:
            print(f"Error in _perturbation: {e}")
            return segment

    @staticmethod
    def create_perturbation_points_by_shuffle(vector_length, samples):
        try:
            return [[random.randint(0, 1) for _ in range(vector_length)] for _ in range(samples)]
        except Exception as e:
            print(f"Error in create_perturbation_points_by_shuffle: {e}")
            return [[0] * vector_length for _ in range(samples)]

    def _convert_perturb_vector_to_traj(self, vector):
        try:
            return sum([
                self.noisy_segments[i] if bit == 1 else self.clean_segments[i]
                for i, bit in enumerate(vector)
            ], [])
        except Exception as e:
            print(f"Error in _convert_perturb_vector_to_traj: {e}")
            return []

    def _perturbed_traj_generator(self):
        try:
            for vector in self.perturb_vectors:
                yield self._convert_perturb_vector_to_traj(vector)
        except Exception as e:
            print(f"Error in _perturbed_traj_generator: {e}")

    def _createZForEval(self):
        try:
            identity_matrix = [[1 if i == j else 0 for j in range(self.x_len)] for i in range(self.x_len)]
            return [
                sum([
                    self.noisy_segments[i] if bit == 1 else self.clean_segments[i]
                    for i, bit in enumerate(row)
                ], [])
                for row in identity_matrix
            ]
        except Exception as e:
            print(f"Error in _createZForEval: {e}")
            return []

    def calc_dtw(self, raw):
        try:
            return [fastdtw(raw, pert, dist=euclidean)[0] for pert in self._perturbed_traj_generator()]
        except Exception as e:
            print(f"Error in calc_dtw: {e}")
            return []

    def _calculate_weight(self):
        try:
            distances = self.calc_dtw(self.X)
            mean_dist = sum(distances) / len(distances)
            std_distances = (sum((x - mean_dist) ** 2 for x in distances) / len(distances)) ** 0.5
            weights = [
                1 if std_distances == 0 else (2.718 ** (-abs((d - mean_dist) / (std_distances + 1e-10))))
                for d in distances
            ]
            return weights
        except Exception as e:
            print(f"Error in _calculate_weight: {e}")
            return []

    def explain(self):
        try:
            Z_trajs = [Trajectory(points=Z_traj) for Z_traj in self._perturbed_traj_generator()]
            labels = [1] * (len(Z_trajs) - 1) + [0]
            Z_pro = Dataset("custom", Z_trajs, labels)
            preds = self.model.predict(Z_pro)
            pred_labels = [pred.argmax() for pred in preds]
            Y = self.model.encoder.inverse_transform(pred_labels)
            
    
            # Kiểm tra nếu Y chỉ có một giá trị duy nhất thì return None
            if len(np.unique(Y)) == 1:
                return None
    
            clf = LogisticRegression()
            clf.fit(self.perturb_vectors, Y, sample_weight=self._calculate_weight())
    
            self.coef_ = clf.coef_
            self.classes_ = clf.classes_
            print(f"coef {self.coef_}")
            print(f"coef {self.classes_}")
            return self.coef_
        except Exception as e:
            print(f"Error in explain: {e}")
            return np.zeros((1, self.x_len))


    def get_Y_eval_sorted(self):
        try:
            Z_trajs = [Trajectory(points=Z_traj) for Z_traj in self.Z_eval]
            labels = [1] * (len(Z_trajs) - 1) + [0]
            Z_pro = Dataset("custom1", Z_trajs, labels)
            Y = self.model.predict(Z_pro)  # Kết quả dự đoán
    
            if Y is None:
                return None  # Nếu Y là None thì return None luôn
    
            Y_without_pertub = self.get_Y()  # Kết quả thực tế, chỉ có 1 class
    
            class_index = None
            if len(np.unique(Y)) > 2:
                # Tìm index của class trong danh sách self.classes_
                class_index = np.where(self.classes_ == Y_without_pertub[0])[0][0]
                print(class_index)
    
                # Lấy hàng trọng số của class tương ứng từ self.coef_
                class_coef = self.coef_[class_index]
            else:
                class_coef = self.coef_[0]
    
            # Sắp xếp chỉ số theo trọng số từ cao đến thấp
            sorted_indices = np.argsort(abs(class_coef))[::-1]
            print(f"sorted_indices for class {self.classes_[class_index]}: {sorted_indices}")
    
            return [Y[i] for i in sorted_indices]
    
        except Exception as e:
            print(f"Error in get_Y_eval_sorted: {e}")
            return np.zeros(len(self.Z_eval))

    def get_Y(self):
        try:
            Z_trajs = [Trajectory(points=self.X)]
            labels = [1] * (len(Z_trajs) - 1) + [0]
            Z_pro = Dataset("custom1", Z_trajs, labels)
            Y = self.model.predict(Z_pro)
            # print(f"Y: {Y}")
            # print(type(Y))
            return Y
        except Exception as e:
            print(f"Error in get_Y: {e}")
            return []

    def get_segment(self):
        try:
            return self.segments
        except Exception as e:
            print(f"Error in get_segment: {e}")
            return []

## Evaluation

### Precision@K

In [7]:
def ap_at_k(y_true, relevant_class, k):
    """
    Calculate AP@K (Average Precision at K) for a given class relevance.

    Parameters:
    - y_true: list of class labels (e.g., ['c1', 'c1', 'c2', 'c1', 'c1', 'c2'])
    - relevant_class: the class to consider as relevant (e.g., 'c1')
    - k: the cut-off rank to consider for AP@K

    Returns:
    - ap_k: Average Precision at K
    """
    if k > len(y_true):
        k = len(y_true)  # Adjust k if it's larger than the list length

    num_relevant = 0  # Count of relevant items encountered so far
    score_sum = 0.0  # Sum of precision at each relevant point
    # print("LOG: y_true", y_true)
    # print("LOG: relevant_class", relevant_class)
    for i in range(k):
        if str(y_true[i]) != relevant_class:
            num_relevant += 1
            precision_at_i = num_relevant / (i + 1)
            score_sum += precision_at_i

    ap_k = score_sum / min(num_relevant, k) if num_relevant > 0 else 0.0
    print("LOG: ap_k", ap_k)
    return ap_k


## Experiment with all trajectories

In [8]:
# LSTM Model
SEED = 0

# datasets = [
#     # Dataset.geolife(),
#     Dataset.animals(),
#     # Dataset.hurdat2(),
#     Dataset.uci_characters(), 
#     Dataset.cma_bst(), 1k
#     # Dataset.mnist_stroke(), 1k
#     # Dataset.uci_pen_digits(), 1k
#     # Dataset.uci_gotrack(),
#     Dataset.uci_movement_libras(),
# ]
# train, test = dataset.mnist_stroke(.8, random_state=SEED)

# Load dataset
dataset = Dataset.uci_movement_libras()

# Split data into train and test subsets
train, test = dataset.split(.8, random_state=SEED)
# train, test = dataset.cut(1000)


# Build and train the model
model = LSTMModel(
    random_state=SEED,
)
# model.init(dataset.classes)
# Train the model on the train dataset
model.train(train, dataset, epochs=10, batch_size=64, checkpoint=None)

# Evaluate the model on a test dataset
evaluation = model.evaluate(test)

# Print the evaluation
evaluation.show()

[==================================================] 100.00%


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 98ms/step - accuracy: 0.0616 - loss: 2.7158
Epoch 2/10
1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - accuracy: 0.0781 - loss: 2.7136

/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.0577 - loss: 2.7219
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - accuracy: 0.0510 - loss: 2.7168
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.0786 - loss: 2.7111
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.0440 - loss: 2.7101
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.0295 - loss: 2.7098
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - accuracy: 0.0595 - loss: 2.7084
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.0595 - loss: 2.7085
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.0648 - loss: 2.7085
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - accuracy: 0.0648 - loss: 2.7086
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 248ms/step

General statistics:

Accuracy: 0.056
F1-score: 0.007
Mean precision: 0.004
Mean recall: 0.067

Confusion matrix:

anti-clockwise arc        circle                    clockwise arc             curved swing              face-

In [9]:
# # ML Model
# SEED = 0

# # datasets = [
# #     # Dataset.geolife(),
# #     Dataset.animals(),
# #     # Dataset.hurdat2(), -> RF cant work
# #     Dataset.cma_bst(),
# #     # Dataset.mnist_stroke(), -> okay
# #     # Dataset.uci_pen_digits(),
# #     # Dataset.uci_gotrack(),
# #     Dataset.uci_characters(),
# #     Dataset.uci_movement_libras(),-> okay
# # ]

# # Load dataset
# dataset = Dataset.geolife()

# # Split data into train and test subsets
# # train, test = dataset.split(.995, random_state=SEED)

# train, test = dataset.filter(
#         lambda traj, label: len(traj) >= 5
#         and traj.r.delta.norm.sum() > 0
#         and dataset.label_counts[label] > 5
#     ).split(
#         train_size=0.985,
#         random_state=SEED,
#     )

# # Build and train the model
# featurizer = featurizers.UniversalFeaturizer()
# model = XGBoostModel(featurizer=featurizer)
# # model = SVMModel(
# #     featurizer=featurizer,
# #     random_state=SEED,
# # )
# # DecisionTreeModel(
# #     featurizer=featurizer,
# #     max_depth=7,
# #     random_state=SEED,
# # # )


# # Train the model on the train dataset
# model.train(train, 5) # 5: CV

# # Evaluate the model on a test dataset
# evaluation = model.evaluate(test)

# # Print the evaluation
# evaluation.show()


In [10]:
import hashlib
import itertools

def generate_unique_name(traj_points):
    """Generate a unique trajectory name based on the hash of trajectory points."""
    try:
        traj_hash = hashlib.md5(str(traj_points).encode()).hexdigest()
        return f"traj_{traj_hash[:8]}"
    except Exception as e:
        print(f"Error generating unique name: {e}")
        return "traj_error"

def experiment(dataset, segment_func, perturbation_func, blackbox_model):
    for traj_idx, (traj, label) in enumerate(zip(dataset.trajs, dataset.labels)):
        try:
            traj_points, traj_label = traj.r, label
            
            if traj_points is None or len(traj_points) == 0:
                print(f"Trajectory {traj_idx} is empty or None. Skipping...")
                continue

            traj_name = generate_unique_name(traj_points)

            try:
                trajectory_experiment = TrajectoryManipulator(traj_points, segment_func, perturbation_func, blackbox_model)
            except Exception as e:
                print(f"Error initializing TrajectoryManipulator for trajectory {traj_idx}: {e}")
                continue

            try:
                coef = trajectory_experiment.explain()
                if coef is None:
                    print("Dont change classification")
                    yield traj_idx, traj_name, 0, 0.0
                    continue
            except Exception as e:
                print(f"Error explaining trajectory {traj_idx}: {e}")
                raise

            try:
                segments = trajectory_experiment.get_segment()
            except Exception as e:
                print(f"Error retrieving segments for trajectory {traj_idx}: {e}")
                continue

            try:
                relevant_class = trajectory_experiment.get_Y()
                if relevant_class is None:
                    print(f"[DEBUG] Prediction failed for trajectory {traj_idx}. Skipping...")
                    continue
            except Exception as e:
                print(f"Error getting ground truth output for trajectory {traj_idx}: {e}")
                continue

            try:
                y_true = trajectory_experiment.get_Y_eval_sorted()
                if y_true is None:
                    print(f"Failed to retrieve label for trajectory {traj_idx}. Skipping...")
                    continue
            except Exception as e:
                print(f"Error retrieving perturbed output for trajectory {traj_idx}: {e}")
                continue

            try:
                y_true_array = y_true[0]
                change = 1 if any(item not in relevant_class for item in y_true) else 0
            except Exception as e:
                print(f"Error computing change for trajectory {traj_idx}: {e}")
                continue

            try:
                precision_score = ap_at_k(y_true, relevant_class, len(y_true)) if change else 0.0
            except Exception as e:
                print(f"Error computing precision score for trajectory {traj_idx}: {e}")
                precision_score = 0.0
                continue
            
            yield traj_idx, traj_name, change, precision_score
        except Exception as e:
            print(f"Unexpected error processing trajectory {traj_idx}: {e}")
            continue

In [11]:
# segment_func = [rdp_segmentation]
# perturbation_func = [gaussian_perturbation]
segment_func = [rdp_segmentation,mdl_segmentation,sliding_window_segmentation]
perturbation_func = [gaussian_perturbation, scaling_perturbation, rotation_perturbation]
# rdp_segmentation
def check_ram_and_log(ram_limit=28, log_dir='logs'):
    """Check RAM usage and log the result."""
    os.makedirs(log_dir, exist_ok=True)
    ram_usage = psutil.virtual_memory().percent
    total_ram = psutil.virtual_memory().total / (1024 ** 3)
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    log_message = f"{timestamp} - Total RAM: {total_ram:.2f} GB | Used: {ram_usage}%\n"
    log_file = os.path.join(log_dir, 'ram_usage_log.txt')
    with open(log_file, 'a') as f:
        f.write(log_message)

    print(log_message.strip())
    return ram_usage > ram_limit

def save_result_row(row, file_path):
    """
    Save a single row of results to a CSV file, ensuring the file and its directory are created if they do not exist.
    """
    # Ensure the directory exists
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    # Initialize a set to track written rows (for deduplication)
    existing_rows = set()

    # Check if file exists
    if os.path.exists(file_path):
        # Read existing rows to prevent duplication
        with open(file_path, mode='r', newline='', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader, None)  # Skip header
            for existing_row in reader:
                existing_rows.add(tuple(existing_row))  # Convert row to tuple
    else:
        # File does not exist; create it with a header
        with open(file_path, mode='w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['index', 'traj_name', 'change', 'precision_score'])

    # Check if the row is already in the file
    if tuple(row) in existing_rows:
        print(f"[INFO] Row already exists in {file_path}: {row}")
        return

    # Append the row to the file
    with open(file_path, mode='a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(row)
        print(f"[INFO] Row saved to {file_path}: {row}")

# Loop through segmentation and perturbation functions
for segment in segment_func:
    for perturbation in perturbation_func:
        # Generate file path for saving results
        file_path = os.path.join('logs', f"{segment.__name__}_{perturbation.__name__}_results.csv")

        # Loop through the experiment results and save row by row
        for traj_idx, traj_name, change, precision_score in experiment(
            dataset=test,
            segment_func=segment,
            perturbation_func=perturbation,
            blackbox_model=model,
        ):
            # Save each row to the CSV
            save_result_row(
                [traj_idx, traj_name, change, precision_score],
                file_path
            )
        print(f"Results saved to {file_path}")


Segments shape: 28, Segments data: [[Vector([0.37524, 0.6412 ])], [Vector([0.37524, 0.64352]), Vector([0.37524, 0.6412 ])], [Vector([0.37524, 0.6412 ])], [Vector([0.37524, 0.64352]), Vector([0.37524, 0.6412 ])], [Vector([0.37524, 0.6412 ])], [Vector([0.37524, 0.6412 ])], [Vector([0.37524, 0.6412 ])], [Vector([0.37524, 0.6412 ])], [Vector([0.37524, 0.6412 ])], [Vector([0.37524, 0.63889]), Vector([0.38685, 0.62269])], [Vector([0.38685, 0.60417])], [Vector([0.37911, 0.60417]), Vector([0.36944, 0.6088 ])], [Vector([0.36557, 0.60417]), Vector([0.36557, 0.58796])], [Vector([0.36557, 0.59028]), Vector([0.34429, 0.61111])], [Vector([0.35397, 0.59954])], [Vector([0.3559, 0.5787])], [Vector([0.35397, 0.58796])], [Vector([0.35783, 0.58796]), Vector([0.3501, 0.5787]), Vector([0.34623, 0.58102]), Vector([0.33849, 0.5787 ]), Vector([0.34043, 0.57639])], [Vector([0.33269, 0.59259])], [Vector([0.34236, 0.58565])], [Vector([0.33656, 0.57407]), Vector([0.33269, 0.57639])], [Vector([0.33462, 0.57639])], 